In [2]:
#%pip -q install google-genai google-adk
!pip install -q google-adk

In [10]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import os
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [11]:
from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [20]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [21]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [50]:
def agente_informacao(consulta):
    informador = Agent(
        name="agente_informacao",
        model=MODEL_ID,
        description="Agente que fornece informações sobre TDAH e TEA.",
        tools=[google_search],
        instruction="""
        Você é uma assistente pessoal especializado em oferecer informações claras e confiáveis sobre
        TDAH (Transtorno do Déficit de Atenção com Hiperatividade) e TEA (Transtorno do Espectro Autista).
        Seu objetivo é responder as perguntas de forma simples, com uma linguagem facil de compreensão e acessível,
        evitando termos médicos complicados. Sempre que possível.
        Suas respostas são baseadas em evidências científicas, estudos médicos e fontes confiáveis.
        Informa no final da mensagem que você é uma assistente virtual. Suas informações não substituem o acompanhamento de um médico,
        psicólogo ou outro profissional de saúde.
        Para um diagnóstico ou tratamento adequado, é fundamental procurar um especialista.
        """,
    )
    resposta = call_agent(informador, consulta)
    return resposta

    """
    Agente para fornecer informações sobre TDAH e TEA.

    Args:
        consulta: A pergunta do usuário sobre TDAH ou TEA.

    Returns:
        A resposta do agente formatada em Markdown.
    """

In [56]:
def agente_estrategias(situacao):
    estrategista = Agent(
        name="agente_estrategias",
        model=MODEL_ID,
        description="Agente que fornece estratégias e dicas para ajudar no dia a dia de pessoas com TDAH e TEA.",
        instruction="""
        Você é um consultor que fornece estratégias práticas e dicas para ajudar pessoas com TDAH e TEA a lidar com desafios do dia a dia.
        Com base na situação ou desafio descrito pelo usuário, forneça sugestões concretas e adaptadas às suas necessidades.
        As estratégias podem incluir técnicas de organização, gerenciamento do tempo, regulação emocional, comunicação, adaptações sensoriais e outras.
        Considere a diversidade de experiências e necessidades dentro do espectro TDAH e TEA.
        Busque informações de fonte confiaveis.
        Lembre que pode funcionar ou não, que cada pessoa tem suas particularidades. Que ele não se sinta mal caso não consiga.
        Informa no final da mensagem que você é uma assistente virtual. Suas não substituem o acompanhamento de um médico,
        psicólogo ou outro profissional de saúde.
        Para um diagnóstico ou tratamento adequado, é fundamental procurar um especialista.
        """,
    )
    resposta = call_agent(estrategista, situacao)
    return resposta

    """
    Agente para fornecer estratégias e dicas para o dia a dia de pessoas com TDAH e TEA.

    Args:
        situacao: A situação ou desafio descrito pelo usuário.

    Returns:
        A resposta do agente formatada em Markdown.
    """

In [57]:
def agente_recursos(localizacao, necessidade):
    buscador_recursos = Agent(
        name="agente_recursos",
        model=MODEL_ID,
        description="Agente que busca recursos e suporte para pessoas com TDAH e TEA.",
        tools=[google_search],
        instruction="""
        Você é um assistente que ajuda pessoas com TDAH e TEA a encontrar recursos e suporte em sua região.
        Com base na localização fornecida pelo usuário, busque informações sobre terapeutas, médicos, grupos de apoio, associações,
        centros de referência e outros serviços relevantes.
        Forneça informações de contato, endereços e, se possível, avaliações ou recomendações.
        Passe endereço e telefone para contato.
        Se a localização não for fornecida, ofereça informações gerais sobre onde buscar ajuda.
        """,
    )
    resposta = call_agent(buscador_recursos, f"Localização: {localizacao}, Necessidade: {necessidade}")
    return resposta

    """
    Agente para buscar recursos e suporte para pessoas com TDAH e TEA.

    Args:
        localizacao: A localização do usuário.
        necessidade: O tipo de recurso ou suporte que o usuário procura.

    Returns:
        A resposta do agente formatada em Markdown.
    """

In [58]:
def agente_apoio(mensagem):
    apoiador = Agent(
        name="agente_apoio",
        model=MODEL_ID,
        description="Agente que oferece apoio emocional e motivação para pessoas com TDAH e TEA.",
        instruction="""
        Você é um amigo virtual que oferece apoio emocional e motivação para pessoas com TDAH e TEA.
        Responda às mensagens dos usuários com empatia, compreensão e encorajamento.
        Valide seus sentimentos, ofereça palavras de esperança e ajude-os a reconhecer suas forças e conquistas.
        Evite dar conselhos médicos ou terapêuticos, mas incentive-os a buscar ajuda profissional quando necessário.
        """,
    )
    resposta = call_agent(apoiador, mensagem)
    return resposta
    """
    Agente para oferecer apoio emocional e motivação para pessoas com TDAH e TEA.

    Args:
        mensagem: A mensagem do usuário.

    Returns:
        A resposta do agente formatada em Markdown.
    """

In [59]:
def agente_mensagem_diaria():
    mensageiro = Agent(
        name="agente_mensagem_diaria",
        model=MODEL_ID,
        description="Agente que fornece uma mensagem motivacional diária para pessoas com TDAH e TEA.",
        instruction="""
        Você é um amigo virtual que fornece uma mensagem motivacional diária para ajudar pessoas com TDAH e TEA a enfrentar seus desafios e ter um dia positivo.
        A mensagem deve ser curta, inspiradora e focada em encorajamento e resiliência.
        Use uma linguagem positiva e afirmativa, destacando a força e a capacidade do usuário de superar obstáculos.
        Evite clichês e frases genéricas, e tente tornar a mensagem o mais pessoal e relevante possível.
        """,
    )
    resposta = call_agent(mensageiro, "Gere uma mensagem motivacional para o dia de hoje.")
    return resposta
    """
    Agente para fornecer uma mensagem motivacional diária.

    Returns:
        A mensagem motivacional formatada em Markdown.
    """

In [61]:
# Função principal para executar a interação com o usuário
def interacao_tdah_tea():
    """
    Função principal para executar a interação com o usuário.
    """
    print("Bem-vindo ao Assistente de Apoio para TDAH e TEA!")

    while True:
        print("\nComo posso te ajudar hoje? \n")
        print("1. Duvidas sobre TDAH ou TEA")
        print("2. Ajuda para o dia a dia")
        print("3. Encontrar recursos e suporte")
        print("4. Receber apoio emocional")
        print("5. Mensagem diária")
        print("6. Sair")

        opcao = input("Digite o número da sua escolha: ")

        if opcao == "1":
            consulta = input("Qual a sua duvida sobre TDAH e TEA? ")
            resposta = agente_informacao(consulta)
            print("\nResposta:")
            display(to_markdown(resposta))

        elif opcao == "2":
            situacao = input("Qual desafio você esta enfrentando? ")
            resposta = agente_estrategias(situacao)
            print("\nEstratégias e dicas:")
            display(to_markdown(resposta))

        elif opcao == "3":
            localizacao = input("Digite sua localização (opcional): ")
            necessidade = input("Digite o tipo de recurso ou suporte que você procura: ")
            resposta = agente_recursos(localizacao, necessidade)
            print("\nRecursos e suporte:")
            display(to_markdown(resposta))

        elif opcao == "4":
            mensagem = input("Como você esta se sentindo hoje? ")
            resposta = agente_apoio(mensagem)
            print("\nApoio emocional:")
            display(to_markdown(resposta))

        elif opcao == "5":
            resposta = agente_mensagem_diaria()
            print("\nMensagem Motivacional Diária:")
            display(to_markdown(resposta))

        elif opcao == "6":
            print("Obrigado por usar o Assistente de Apoio. Até a próxima!")
            break

        else:
            print("Opção inválida. Por favor, tente novamente.")

# Execução da função principal
if __name__ == "__main__":
    interacao_tdah_tea()

Bem-vindo ao Assistente de Apoio para TDAH e TEA!

Como posso te ajudar hoje? 

1. Duvidas sobre TDAH ou TEA
2. Ajuda para o dia a dia
3. Encontrar recursos e suporte
4. Receber apoio emocional
5. Mensagem diária
6. Sair
Digite o número da sua escolha: 6
Obrigado por usar o Assistente de Apoio. Até a próxima!
